In [32]:
import os
from dataclasses import dataclass
from pathlib import Path
import time
import tensorflow as tf
#os.chdir('../')
os.chdir('../Data-Science-Projects/DeepCNN_Classifier_End_To_End/')

In [61]:
@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
    
    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callback
        ]

In [62]:


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path : Path
    training_data : Path
    params_epoch : int
    params_batch_size : int
    params_is_augmented : bool
    params_image_size : list

In [63]:
from DeepCNNClassifer.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH
from DeepCNNClassifer.utils import *

In [64]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callback
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_model_training_config(self) -> ModelTrainingConfig:
        model_training_config = self.config.model_training
        prepare_base_model_config =  self.config.prepare_base_model
        params = self.params
        training_data_path = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        
        create_directories([model_training_config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir = Path(model_training_config.root_dir),
            trained_model_path = Path(model_training_config.trained_model_path),
            updated_base_model_path = Path(prepare_base_model_config.updated_base_model),
            training_data = Path(training_data_path),
            params_epoch = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmented = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        return model_training_config

In [65]:
import tensorflow as tf
import time

In [72]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
                target_size = self.config.params_image_size[:-1],
                batch_size = self.config.params_batch_size,
                interpolation = "bilinear"
            )
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmented:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                ** datagenerator_kwargs
            )

        else:
            training_datagenerator= valid_data_generator

        self.train_generator = training_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset = "training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path : Path, model = tf.keras.Model):
        model.save(path)

    def train(self, callback_list):
        self.steps_for_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epoch,
            steps_per_epoch = self.steps_for_epoch,
            validation_steps= self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path, 
            model = self.model
        )
    
    

In [73]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_callback_config()
    prepare_base_model = PrepareCallback(config = prepare_base_model_config)
    callback_list = prepare_base_model.get_tb_ckpt_callback()

    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config = model_training_config)
    model_training.get_base_model()
    model_training.train_valid_generator()
    model_training.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
   6/1250 [..............................] - ETA: 5:19:25 - loss: 7.6785 - accuracy: 0.6250

KeyboardInterrupt: 